In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
print('一共有 {} 个训练集样本'.format(train.shape[0]))
print('一共有 {} 个测试集样本'.format(test.shape[0]))

一共有 3863 个训练集样本
一共有 497 个测试集样本


In [8]:
all_features = pd.concat((train.loc[:, '21000':'350014'],
                          test.loc[:, '21000':'350014']))

In [9]:
numeric_feats = all_features.dtypes[all_features.dtypes != "object"].index # 取出所有的数值特征

# 减去均值，除以方差
all_features[numeric_feats] = all_features[numeric_feats]#.apply(lambda x: (x - x.mean())/ (x.std())) 
                                                                

In [22]:
train.loc[:,'x':'y']

,x,y
0,0.0,10.8
1,0.0,11.4
2,0.0,12.0
3,0.0,12.6
4,0.0,13.2
5,0.0,13.8
6,0.0,14.4
7,0.0,15.0
8,0.0,15.6
9,0.0,16.2


In [11]:
feat_dim = all_features.shape[1]

In [48]:
num_train = int(0.9 * train.shape[0]) # 划分训练样本和验证集样本
indices = np.arange(train.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

# 提取训练集和验证集的特征
train_features = all_features.iloc[train_indices].values.astype(np.float32)
train_features = torch.from_numpy(train_features)
valid_features = all_features.iloc[valid_indices].values.astype(np.float32)
valid_features = torch.from_numpy(valid_features)
train_valid_features = all_features[:train.shape[0]].values.astype(np.float32)
train_valid_features = torch.from_numpy(train_valid_features)
test_features = all_features[train.shape[0]:].values.astype(np.float32)
test_features = torch.from_numpy(test_features)

# 提取训练集和验证集的label
train_labels_x = train['x'].values[train_indices, None].astype(np.float32)
train_labels_y = train['y'].values[train_indices, None].astype(np.float32)
train_labels   = np.hstack([train_labels_x,train_labels_y])
train_labels = torch.from_numpy(train_labels)

valid_labels_x = train['x'].values[valid_indices, None].astype(np.float32)
valid_labels_y = train['y'].values[valid_indices, None].astype(np.float32)
valid_labels   = np.hstack([valid_labels_x,valid_labels_y])
valid_labels = torch.from_numpy(valid_labels)

train_valid_labels_x = train['x'].values[:, None].astype(np.float32)
train_valid_labels_y = train['y'].values[:, None].astype(np.float32)
train_valid_labels   = np.hstack([train_valid_labels_x,train_valid_labels_y])
train_valid_labels = torch.from_numpy(train_valid_labels)

#train_valid_labels_x = torch.from_numpy(train_valid_labels_x)
#train_valid_labels_y = torch.from_numpy(train_valid_labels_y)



In [68]:
def get_model():
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim/2)),
        nn.BatchNorm1d(int(feat_dim/2)),
        nn.ReLU(inplace=True),
        nn.Linear(int(feat_dim/2),int(feat_dim/4)),
        nn.BatchNorm1d(int(feat_dim/4)),
        nn.ReLU(inplace=True),
        nn.Linear(int(feat_dim/4), 2)
    )
    return net

net = get_model()
print(net)

Sequential(
  (0): Linear(in_features=30, out_features=15, bias=True)
  (1): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): Dropout(p=0.5)
  (4): Linear(in_features=15, out_features=7, bias=True)
  (5): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace)
  (7): Dropout(p=0.5)
  (8): Linear(in_features=7, out_features=2, bias=True)
)


# train and predict

In [95]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)


In [96]:
error(a,b)

1.4142135623730951

history


In [ ]:
log
